In [1]:
import os

import pandas as pd
import numpy as np

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [2]:
abt = pd.read_csv("../Dataset/analytical_base_table.csv")
abt.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Separate dataframe into separate object

# Object for target variable
y = abt.Exited

# object for input features
X = abt.drop(['Exited'], axis=1)

# display shapes of X and y
print(X.shape, y.shape)

(10000, 10) (10000,)


In [4]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [5]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['Geography', 'Gender']

In [6]:
def class_count(a):
    counter=Counter(a)
    kv=[list(counter.keys()),list(counter.values())]
    abt2 = pd.DataFrame(np.array(kv).T, columns=['Exited','Count'])
    abt2['Count'] = abt2['Count'].astype('int64')
    abt2['%'] = round(abt2['Count'] / a.shape[0] * 100, 2)
    return abt2.sort_values('Count',ascending=False)

class_count(y)

,Exited,Count,%
1,0,7963,79.63
0,1,2037,20.37


In [7]:
random_state = 10

# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=abt.Exited)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

7000 3000 7000 3000


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 8061 to 4741
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      7000 non-null   int64  
 1   Geography        7000 non-null   object 
 2   Gender           7000 non-null   object 
 3   Age              7000 non-null   int64  
 4   Tenure           7000 non-null   int64  
 5   Balance          7000 non-null   float64
 6   NumOfProducts    7000 non-null   int64  
 7   HasCrCard        7000 non-null   int64  
 8   IsActiveMember   7000 non-null   int64  
 9   EstimatedSalary  7000 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 601.6+ KB


## **Performing Data Preprocessing - Pipeline**

<h5> Scale numerical data and encode categorical data
Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder

<h5> Create lists of indexes from the list of column names

<h5> Need to be numeric not string to specify columns name in column transformer

In [9]:
num_features = [] 

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features) 

[0, 3, 4, 5, 6, 7, 8, 9]


In [10]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features) 

[1, 2]


In [11]:
# Define column transformer
# Need to be numeric not string to specify columns name 
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse=False), cat_features)
)
preprocess

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [0, 3, 4, 5, 6, 7, 8, 9]),
                                ('onehotencoder', OneHotEncoder(sparse=False),
                                 [1, 2])])

## **Build Model Pipeline without SMOTE**
<h5> To see the impact of SMOTE to our results, wew will first build our pipeline without SMOTE

In [12]:
# Import classifier
from sklearn.svm import SVC 

# Define model with pipeline
model = imbl_pipe(preprocess,
                  SVC(random_state=random_state))

model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  [1, 2])])),
                ('svc', SVC(random_state=10))])

In [13]:
# Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
              'svc__C': [0.0005,0.001, 0.01, 0.1, 0.5],
              'svc__gamma': [5, 1, 0.1, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3, cv= 5, n_jobs=4)

In [14]:
X_train = X_train.values
X_test = X_test.values


In [15]:
# Train the model with GridSearch
grid.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarn

[CV 4/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.796 total time=   1.0s
[CV 3/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.796 total time=   1.1s


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 2/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.796 total time=   1.5s
[CV 1/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.796 total time=   1.5s


...

/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 2/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.796 total time=   3.9s


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 3/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.796 total time=   3.7s
[CV 4/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.796 total time=   3.7s
[CV 5/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.796 total time=   3.6s
[CV 5/5] END svc__C=0.5, svc__gamma=5, svc__kernel=poly;, score=0.857 total time= 3.2min


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('minmaxscaler',
                                                                         MinMaxScaler(),
                                                                         [0, 3,
                                                                          4, 5,
                                                                          6, 7,
                                                                          8,
                                                                          9]),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(sparse=False),
                                                                         [1,
                                                                          2])])),
                                       ('svc', SVC(random_state=10))]),
             n_jobs=4,
             param_grid={'svc__C': [0.0005, 0.001, 0.01, 0.1, 0.5],
                         'svc__gamma': [5, 1, 0.1, 0.01],
                         'svc__kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             verbose=3)

In [16]:
print(grid.best_params_)

{'svc__C': 0.5, 'svc__gamma': 5, 'svc__kernel': 'poly'}


In [17]:
 # List the best score
print(grid.best_score_)

0.8551428571428572


In [18]:
print(f"Training Data Score: {grid.score(X_train, y_train)}")
print(f"Testing Data Score: {grid.score(X_test, y_test)}")

Training Data Score: 0.875
Testing Data Score: 0.8646666666666667


In [20]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, predictions)
print(cm)

[[2311   78]
 [ 328  283]]


In [22]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

[[0.97 0.03]
 [0.54 0.46]]


In [23]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      2389
           1       0.78      0.46      0.58       611

    accuracy                           0.86      3000
   macro avg       0.83      0.72      0.75      3000
weighted avg       0.86      0.86      0.85      3000



In [24]:
predictions

array([0, 0, 0, ..., 0, 0, 0])

In [25]:
pred = grid.predict(X_test[:1])

In [26]:
print(f"Predicted classes: {pred}")
print(f"Actual Labels: {list(y_test[:1])}")

Predicted classes: [0]
Actual Labels: [1]


## **Build Model Pipeline with SMOTE**
<h4> Let's check now the impact of SMOTE to our results

<h6> We are going to use the Pipeline from the imblearn package in place of scikit-learn Pipeline.

<h6> It takes care automatically to re-sample when called fit() on the pipeline, and does not re-sample test data (when called transform() or predict()).

In [27]:
# Import classifier
from sklearn.svm import SVC 

# Define model with pipeline
model_s = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  SVC(random_state=random_state))

model_s

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 3, 4, 5, 6, 7, 8, 9]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  [1, 2])])),
                ('smote', SMOTE(random_state=10)),
                ('svc', SVC(random_state=10))])

In [28]:
# Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV

param_grid = {'svc__kernel' : ['linear', 'rbf', 'poly', 'sigmoid'],
              'svc__C': [0.0005,0.001, 0.01, 0.1, 0.5],
              'svc__gamma': [5, 1, 0.1, 0.01]}
grid_s = GridSearchCV(model_s, param_grid, verbose=3, cv= 5, n_jobs=4)

In [29]:
# Train the model with GridSearch
grid_s.fit(X_train, y_train)

Fitting 5 folds for each of 80 candidates, totalling 400 fits


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarn

[CV 2/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.677 total time=   6.9s


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 4/5] END svc__C=0.0005, svc__gamma=5, svc__kernel=linear;, score=0.690 total time=   7.2s


...

[CV 1/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.664 total time=   6.7s


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 2/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.678 total time=   6.6s


/home/pikosho/.conda/envs/bank_churn/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


[CV 3/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.684 total time=   6.6s
[CV 4/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.673 total time=   6.5s
[CV 5/5] END svc__C=0.5, svc__gamma=0.01, svc__kernel=sigmoid;, score=0.689 total time=   6.5s
[CV 5/5] END svc__C=0.5, svc__gamma=5, svc__kernel=poly;, score=0.784 total time= 4.4min


In [1]:
print(grid_s.best_params_)

NameError: name 'grid_s' is not defined

In [ ]:
 # List the best score
print(grid_s.best_score_)

In [ ]:
print(f"Training Data Score: {grid_s.score(X_train, y_train)}")
print(f"Testing Data Score: {grid_s.score(X_test, y_test)}")

In [ ]:
# Make predictions with the hypertuned model
predictions_s = grid_s.predict(X_test)
predictions_s


In [ ]:
# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, predictions_s)
print(cm)

In [ ]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

In [ ]:
print(classification_report(y_test, predictions_s))

In [ ]:
predictions_s

In [ ]:
pred_s = grid.predict(X_test[:1])

In [ ]:
print(f"Predicted classes: {pred_s}")
print(f"Actual Labels: {list(y_test[:1])}")

<h5> 

<h5> Saving the model

In [ ]:
import joblib

# We are saving our grid model
filename_nos = '../SAV files/SVM_model_nos.sav'
joblib.dump(grid, filename_nos)
filename_s = '../SAV files/SVM_model_s.sav'
joblib.dump(grid_s, filename_s)

In [ ]:
# load the models
svm_model_nos = joblib.load(filename_nos)
print(svm_model_nos.score(X_test, y_test))

svm_model_s = joblib.load(filename_s)
print(svm_model_s.score(X_test, y_test))